In [1]:
from torchvision import datasets, transforms, models
from torch import nn, optim, utils, device as device_, cuda
import torch
import torch.nn as nn
import numpy as np
from sklearn import metrics
import time
import sparseconvnet as scn
import math

In [2]:
dataset_train = datasets.MNIST(
    './data', 
    train=True, 
    download=True, 
    transform=transforms.ToTensor())

In [175]:
use_cuda = torch.cuda.is_available() and scn.SCN.is_cuda_build()
device = 'cuda:0' if use_cuda else 'cpu'
if use_cuda:
    print("Using CUDA.")
else:
    print("Not using CUDA.")

Using CUDA.


In [176]:
class Data(torch.utils.data.Dataset):
    def __init__(self,file):
        torch.utils.data.Dataset.__init__(self)
        
        total_pix = 0.0
        data_pix = 0.0
        
        N_DATA = 100
        
        data = torch.load(file)
        self.data = []
        # self.data = data[0] # array of img tensors
        
        self.labels = data[1]
        
#         self.labels = [] # array of labels
#         for label in data[1]:
#             self.labels.append(torch.nn.functional.one_hot(label, 10).to(device))
        
        for batchIdx, img in enumerate(data[0]): # for each data
            
            if batchIdx >= N_DATA: break
                
            locations = []
            features = []
            for y, line in enumerate(img): # for y coordinates
                for x, v in enumerate(line): # for x coordinates
                    total_pix += 1
                    if v != 0:
                        locations.append([y, x, batchIdx])
                        features.append([v])
                        data_pix += 1
            
            single_data = {}
            single_data['locations'] = torch.LongTensor(locations)
            single_data['features'] = torch.FloatTensor(features).to(device)
            self.data.append(single_data)
        
        print('Sparse ', data_pix / total_pix)
        print('Loaded ', len(self.data), ' points')
                
    
    def __getitem__(self, n):
        return self.data[n], self.labels[n]

    def __len__(self):
        return len(self.data)

In [177]:
dataset = Data('data/MNIST/processed/training.pt')

Sparse  0.1865561224489796
Loaded  100  points


In [184]:
dataloader_train = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [264]:
model = scn.Sequential().add(
    scn.SparseVggNet(2, 1,
                     [['C', 8], ['C', 8], ['MP', 3, 2],
                      ['C', 16], ['C', 16], ['MP', 3, 2],
                      ['C', 24], ['C', 24], ['MP', 3, 2]])
).add(
    scn.SubmanifoldConvolution(2, 24, 10, 3, False)
).add(
    scn.BatchNormReLU(10)
).add(
    scn.SparseToDense(2, 10)
).to(device)

inputSpatialSize = model.input_spatial_size(torch.LongTensor([1]))
input_layer = scn.InputLayer(2, inputSpatialSize)

class MyNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        inputSpatialSize = model.input_spatial_size(torch.LongTensor([1]))
        self.inputLayer = scn.InputLayer(2, inputSpatialSize)
        
#         self.sparseModel = scn.Sequential().add(
#                 scn.SparseVggNet(2, 1,
#                                  [['C', 8], ['C', 8], ['MP', 3, 2],
#                                   ['C', 16], ['C', 16], ['MP', 3, 2],
#                                   ['C', 24], ['C', 24], ['MP', 3, 2]])
#             ).add(
#                 scn.SubmanifoldConvolution(2, 24, 10, 3, False)
#             ).add(
#                 scn.BatchNormReLU(10)
#             ).add(
#                 scn.SparseToDense(2, 10)
#             )

        self.spaseModel = scn.Sequential().add(
            scn.SubmanifoldConvolution(2, 32, 3, 1, False)
            .add(
                scn.BatchNormReLU()
            ).add(
        )
    
    def forward(self, x):
        #print("forward:", len(x))
        # x = self.input(x)
        x = self.sparseModel(x)
        # x = torch.reshape(x, (1, 10))
        #x = torch.softmax(torch.reshape(x, (1, 10)), 1)
        # print(x)
        # x = nn.Softmax(x, 0)
        return x


class MyLoss(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, output, target):
        
        loss = torch.mean((output - target)**2)
        return loss

In [265]:
model = MyNet().cuda()

EPOCHS = 100

# criterion = torch.nn.CrossEntropyLoss()
criterion = MyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8)

In [266]:
for epoch in range(EPOCHS):
    
    model.train()
    train_loss = 0
    
    n_correct = 0
    n_total = 0
    
    #for data, label in dataset:
    for i,batch in enumerate(dataset):
        data = batch[0]
        label = batch[1]
        optimizer.zero_grad()
        input = input_layer([data['locations'],data['features']])
        output = model(input)[i]
        loss = criterion(output, label)
        # loss = criterion(torch.reshape(output, (1, 10)), label)
        train_loss += loss
        loss.backward()
        optimizer.step()
        
        # print(model[0][0].weight.grad)
        
        if torch.argmax(output) == label:
            n_correct += 1
        n_total += 1
    
        print(f'epoch: {epoch+1:4}, loss: {train_loss}, accuracy: {n_correct / n_total}')

epoch:    1, loss: 24.999984741210938, accuracy: 0.0


KeyboardInterrupt: 

In [258]:
torch.softmax(torch.reshape(output, (1, 10)), 1)

tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000]], device='cuda:0', grad_fn=<SoftmaxBackward>)

In [257]:
torch.reshape(output, (1, 10))

tensor([[1.1921e-07, -0.0000e+00, 1.6689e-06, -0.0000e+00, 1.1921e-07, -0.0000e+00,
         4.7684e-07, 0.0000e+00, 5.9605e-07, 3.5763e-06]], device='cuda:0',
       grad_fn=<ViewBackward>)

In [227]:
torch.softmax(torch.Tensor([1, 2, 3]), 0)

tensor([0.0900, 0.2447, 0.6652])

In [223]:
torch.Tensor([1, 2, 3])

tensor([1., 2., 3.])

In [267]:
output

tensor([[[1.4305e-06]],

        [[2.1458e-06]],

        [[-0.0000e+00]],

        [[-0.0000e+00]],

        [[1.9073e-06]],

        [[1.5497e-06]],

        [[6.1989e-06]],

        [[2.3842e-06]],

        [[-0.0000e+00]],

        [[-0.0000e+00]]], device='cuda:0', grad_fn=<SelectBackward>)